In [2]:
import os
os.environ["JAVA_HOME"] = "C:\\jdk-22.0.2"
os.environ["SPARK_HOME"] = "C:\\spark-3.5.2-bin-hadoop3\\spark-3.5.2-bin-hadoop3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_PYTHON"] = "python"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, lit, regexp_extract
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from sklearn.metrics import roc_curve, auc
import pandas as pd
import numpy as np

In [4]:
spark = SparkSession.builder.appName('Pyspark_Titanic').getOrCreate()

In [5]:
train = spark.read.csv('C:\\01-FaculdadeSemestreAtual\\Processamento_de_Dados_Massivos\\Loan_Approval\\train.csv',header = 'True',inferSchema='True')
test = spark.read.csv('C:\\01-FaculdadeSemestreAtual\\Processamento_de_Dados_Massivos\\Loan_Approval\\test.csv',header = 'True',inferSchema='True')

In [13]:
train.show(5)

+---+----------+-------------+---------------------+-----------------+-----------+----------+---------+-------------+-------------------+-------------------------+--------------------------+-----------+
| id|person_age|person_income|person_home_ownership|person_emp_length|loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|loan_status|
+---+----------+-------------+---------------------+-----------------+-----------+----------+---------+-------------+-------------------+-------------------------+--------------------------+-----------+
|  0|        37|        35000|                 RENT|              0.0|  EDUCATION|         B|     6000|        11.49|               0.17|                        N|                        14|          0|
|  1|        22|        56000|                  OWN|              6.0|    MEDICAL|         C|     4000|        13.35|               0.07|                        N|                         

In [14]:
train.count()

58645

In [7]:
train.printSchema()

root
 |-- id: integer (nullable = true)
 |-- person_age: integer (nullable = true)
 |-- person_income: integer (nullable = true)
 |-- person_home_ownership: string (nullable = true)
 |-- person_emp_length: double (nullable = true)
 |-- loan_intent: string (nullable = true)
 |-- loan_grade: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- loan_int_rate: double (nullable = true)
 |-- loan_percent_income: double (nullable = true)
 |-- cb_person_default_on_file: string (nullable = true)
 |-- cb_person_cred_hist_length: integer (nullable = true)
 |-- loan_status: integer (nullable = true)



In [8]:
train.select([count(when(col(c).isNull(), c)).alias(c) for c in train.columns]).show()

+---+----------+-------------+---------------------+-----------------+-----------+----------+---------+-------------+-------------------+-------------------------+--------------------------+-----------+
| id|person_age|person_income|person_home_ownership|person_emp_length|loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|loan_status|
+---+----------+-------------+---------------------+-----------------+-----------+----------+---------+-------------+-------------------+-------------------------+--------------------------+-----------+
|  0|         0|            0|                    0|                0|          0|         0|        0|            0|                  0|                        0|                         0|          0|
+---+----------+-------------+---------------------+-----------------+-----------+----------+---------+-------------+-------------------+-------------------------+-------------------------

In [9]:
train.limit(100).toPandas().sample(10)

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
81,81,30,45600,RENT,6.0,DEBTCONSOLIDATION,A,23500,7.66,0.50,N,8,1
80,80,26,60000,RENT,2.0,DEBTCONSOLIDATION,B,7000,10.62,0.12,N,3,0
76,76,22,70000,MORTGAGE,4.0,DEBTCONSOLIDATION,A,10000,7.29,0.14,N,2,0
22,22,26,95000,MORTGAGE,10.0,EDUCATION,A,10000,7.68,0.11,N,3,0
82,82,24,75000,RENT,3.0,PERSONAL,B,15000,11.49,0.20,N,2,0
74,74,23,100000,MORTGAGE,7.0,EDUCATION,A,6000,7.74,0.06,N,3,0
15,15,29,33000,OWN,8.0,MEDICAL,A,7300,8.90,0.23,N,8,0
47,47,25,39996,RENT,2.0,DEBTCONSOLIDATION,B,8000,12.53,0.20,N,4,0
36,36,24,90000,MORTGAGE,2.0,DEBTCONSOLIDATION,A,14400,7.90,0.16,N,2,0
14,14,27,100000,RENT,1.0,HOMEIMPROVEMENT,C,5000,13.57,0.05,Y,7,0


In [10]:
train.groupby('loan_intent').count().show()

+-----------------+-----+
|      loan_intent|count|
+-----------------+-----+
|DEBTCONSOLIDATION| 9133|
|          VENTURE|10011|
|         PERSONAL|10016|
|        EDUCATION|12271|
|  HOMEIMPROVEMENT| 6280|
|          MEDICAL|10934|
+-----------------+-----+



In [11]:
train.groupby('loan_grade').count().show()

+----------+-----+
|loan_grade|count|
+----------+-----+
|         F|  149|
|         E| 1009|
|         B|20400|
|         D| 5034|
|         C|11036|
|         A|20984|
|         G|   33|
+----------+-----+

